# 케라스로 RNN 구현하기

### SimpleRNN()

In [18]:
import tensorflow as tf

In [19]:
tf.__version__

'2.8.2'

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

### SimpleRNN 모델의 출력 결과

**1) 출력값이 (batch_size, output_dim) 크기의 2D 텐서이고 입력을 (timesteps, input_dim)로 정의할 때**

In [21]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_10 (SimpleRNN)   (None, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


**2) 출력값이 (batch_size, output_dim) 크기의 2D 텐서이고 입력을 (batch_size, timesteps, input_dim)로 정의할 때**

In [22]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_11 (SimpleRNN)   (8, 3)                    42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


**3) 출력값이 (batch_size, timesteps, output_dim) 크기의 3D 텐서이고 입력을 (batch_size, timesteps, input_dim)로 정의할 때**

In [23]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_12 (SimpleRNN)   (8, 2, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


# 파이썬으로 RNN 구현하기

### 직접 Numpy로 RNN 층 구현하기

**1) 시점의 수(문장의 길이)인 timesteps, 입력의 차원(단어 벡터의 차원)인 input_dim, 은닉 상태의 크기(메모리 셀의 용량)인 hidden_units을 정의**

In [24]:
import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_dim = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다.

inputs = np.random.random((timesteps, input_dim)) # 입력에 해당되는 2D 텐서

# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬.
hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태는 0(벡터)로 초기화

**2) 초기 은닉 상태를 0의 값을 가지는 벡터로 초기화 후 초기 은닉 상태 출력**

In [25]:
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬.# 8의 크기를 가지는 은닉 상태. 현재는 초기 은닉 상태로 모든 차원이 0의 값을 가짐.
print(hidden_state_t)

[0. 0. 0. 0. 0. 0. 0. 0.]


**3) 가중치와 편향을 각 크기에 맞게 정의하고 크기를 출력**

In [26]:
Wx = np.random.random((hidden_size, input_dim))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_size,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

In [27]:
print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


**4) 모든 시점의 은닉 상태를 출력한다고 가정하고 RNN 층을 동작**

In [28]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) # Wx * Xt + Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적
  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0) 
# 출력 시 값을 깔끔하게 해준다.

print(total_hidden_states) # (timesteps, output_dim)의 크기. 이 경우 (10, 8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력.

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.87937286 0.68386273 0.85489811 0.97672323 0.9378183  0.93731179
  0.91001751 0.49493412]
 [0.99975327 0.99991562 0.99909372 0.99996822 0.9989128  0.99961658
  0.99952821 0.99478822]
 [0.99990703 0.9999739  0.99978966 0.999995   0.99978803 0.99993293
  0.99988482 0.99810895]
 [0.99996831 0.99999029 0.99973396 0.99999764 0.99988742 0.99997447
  0.99994888 0.99943586]
 [0.99995485 0.99998795 0.9997375  0.99999721 0.99987538 0.99998323
  0.99996569 0.99957187]
 [0.99991957 0.99997653 0.99980672 0.99999626 0.99982246 0.9999576
  0.99992512 0.99865708]
 [0.99985723 0.9999714  0.99960051 0.99998456 0.99960412 0.99989695
  0.99980414 0.99825034]
 [0.99997767 0.99999425 0.99983912 0.9999992  0.99996875 0.99998749
  0.99997944 0.99962306]
 [0.99996837 0.99999256 0.99972453 0.99999813 0.99992565 0.99999181
  0.99998277 0.99979865]
 [0.99995703 0.99998618 0.99973924 0.9999969  0.9998375  0.99996933
  0.99993777 0.99926509]]

# 깊은 순환 신경망

In [29]:
model = Sequential()
model.add(SimpleRNN(hidden_size, input_length=10, input_dim=5, return_sequences = True))
model.add(SimpleRNN(hidden_size, return_sequences = True))
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_13 (SimpleRNN)   (None, 10, 8)             112       
                                                                 
 simple_rnn_14 (SimpleRNN)   (None, 10, 8)             136       
                                                                 
Total params: 248
Trainable params: 248
Non-trainable params: 0
_________________________________________________________________


# 양방향 순환 신경망

In [30]:
from tensorflow.keras.layers import Bidirectional

In [31]:
timesteps = 10
input_dim = 5

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_5 (Bidirectio  (None, 10, 16)           224       
 nal)                                                            
                                                                 
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________


In [32]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_6 (Bidirectio  (None, 10, 16)           224       
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 10, 16)           400       
 nal)                                                            
                                                                 
 bidirectional_8 (Bidirectio  (None, 10, 16)           400       
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 10, 16)           400       
 nal)                                                            
                                                                 
Total params: 1,424
Trainable params: 1,424
Non-train

# 케라스의 SimpleRNN 이해하기

### 임의의 입력 생성하기

**1) RNN 테스트를 위한 임의의 입력을 생성**

In [33]:
train_X = [[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]
print(np.shape(train_X))

(4, 5)


**2) RNN이 입력받도록 하기 위해 입력으로 만든 2D 텐서를 3D 텐서로 변경되도록 배치 크기 1을 추가**

In [34]:
train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape)

(1, 4, 5)


### SimpleRNN 이해하기

**1) return_seqeunces와 return_state 인자를 둘 다 False로 지정한 후 출력하기**

In [35]:
rnn = SimpleRNN(3)
# rnn = SimpleRNN(3, return_sequences=False, return_state=False)와 동일.
hidden_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))

hidden state : [[-0.44236007 -0.6771831  -0.9898478 ]], shape: (1, 3)


**2) return_sequences 인자만 True로 지정한 후 출력하기**

In [36]:
rnn = SimpleRNN(3, return_sequences=True)
hidden_states = rnn(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))

hidden states : [[[-0.85783446  0.39400023 -0.8971302 ]
  [-0.616635    0.8943555  -0.5897024 ]
  [-0.5597038   0.32040784 -0.86222243]
  [-0.22487181  0.5836284  -0.84703285]]], shape: (1, 4, 3)


**3) return_seqeunces와 return_state 인자를 둘 다 True로 지정한 후 출력하기**

In [37]:
rnn = SimpleRNN(3, return_sequences=True, return_state=True)
hidden_states, last_state = rnn(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))

hidden states : [[[-0.9189601  -0.998121   -0.5466572 ]
  [-0.6043672  -0.60413235 -0.8970105 ]
  [ 0.33515498 -0.8971837  -0.45518568]
  [-0.914966   -0.80071056  0.6213814 ]]], shape: (1, 4, 3)
last hidden state : [[-0.914966   -0.80071056  0.6213814 ]], shape: (1, 3)


**4) return_state 인자만 True로 지정한 후 출력하기**

In [38]:
rnn = SimpleRNN(3, return_sequences=False, return_state=True)
hidden_state, last_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))

hidden state : [[-0.9173524   0.65808797 -0.99765325]], shape: (1, 3)
last hidden state : [[-0.9173524   0.65808797 -0.99765325]], shape: (1, 3)
